In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.datasets import load_boston
import random

# machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# preprocessing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# auxiliares modelagem
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, RFE

# importando funções helpers
from helpers import cross_validation


warnings.filterwarnings('ignore')

plt.style.use('seaborn-pastel')

%matplotlib inline

# 1- Importando os dados

In [2]:
df = pd.read_csv('data/boston_house_prices_clean.csv')

In [3]:
X_train = df.query('set == "treino"')
X_test = df.query('set == "test"')

In [4]:
y_train = X_train['target']
y_test = X_test['target']

In [5]:
X_train.drop(['target', 'set'], axis=1, inplace=True)
X_test.drop(['target', 'set'], axis=1, inplace=True)

# 2 - Modelagem

### 2.1 - OLS - Ordinary Least Squares

In [6]:
cols = [
    'CRIM', 
    'ZN', 
    #'INDUS', 
    'CHAS', 
    'NOX', 
    'RM', 
    #'AGE', 
    'DIS', 
    'RAD', 
    'TAX', 
    'PTRATIO', 
    'B', 
    'LSTAT'
]
X_train_ = sm.add_constant(X_train[cols])
model = sm.OLS(y_train, X_train_)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     114.8
Date:                Wed, 21 Apr 2021   Prob (F-statistic):          3.27e-115
Time:                        16:50:18   Log-Likelihood:                -1180.1
No. Observations:                 404   AIC:                             2384.
Df Residuals:                     392   BIC:                             2432.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.6119      0.227     99.682      0.000      22.166      23.058
CRIM          -0.9193      0.302     -3.040      0.003      -1.514      -0.325
ZN             0.9072      0.352      2.575      0.010       0.215       1.600
CHAS           0.5789      0.233      2.485      0.013       0.121       1.037
NOX           -1.9324      0.466     -4.150      0.000      -2.848      -1.017
RM             2.1805      0.314      6.952      0.000       1.564       2.797
DIS           -2.8510      0.410     -6.949      0.000      -3.658      -2.044
RAD            2.4399      0.561      4.348      0.000       1.337       3.543
TAX           -2.0301      0.586     -3.467      0.001      -3.181      -0.879
PTRATIO       -2.3134      0.301     -7.680      0.000      -2.906      -1.721
B              0.6606      0.264      2.505      0.013       0.142       1.179
LSTAT         -4.0238      0.365    -11.026      0.000      -4.741      -3.306
==============================================================================
Omnibus:                      111.492   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              349.433
Skew:                           1.255   Prob(JB):                     1.32e-76
Kurtosis:                       6.803   Cond. No.                         7.48
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Testando a performance do modelo no conjunto de testes

In [7]:
print("MSE no Treino: {}".format(mean_squared_error(y_train, results.predict())))

MSE no Treino: 20.17083868132372


In [8]:
X_test_ = sm.add_constant(X_test[cols])

print("R2 no Teste: {}".format(r2_score(y_test, results.predict(X_test_))))
print("MSE no Teste: {}".format(mean_squared_error(y_test, results.predict(X_test_))))

R2 no Teste: 0.5267027332737364
MSE no Teste: 38.539857641783044


# 2.2 - Modelo Linear

In [9]:
# criando modelo de regressão linear
lr = LinearRegression()

# validação cruzada
kf = KFold(n_splits=5)

cross_validation(lr, kf, X_train, y_train)

MSE no training: 19.962089480274948 +- 1.071692734938361
MSE no validation: 21.99430996885578 +- 4.213772938343666
r2 no training: 0.7653848730459873 +- 0.009351851630105544
r2 no validation: 0.7377105814926351 +- 0.037821332801799556


In [10]:
print("R2 no Teste: {}".format(r2_score(y_test, lr.predict(X_test))))
print("MSE no Teste: {}".format(mean_squared_error(y_test, lr.predict(X_test))))

R2 no Teste: 0.5091943549605271
MSE no Teste: 39.965537558332876


# 2.3 - Modelo de distância

In [11]:
knn = KNeighborsRegressor(n_neighbors=5)

cross_validation(knn, kf, X_train, y_train)

MSE no training: 14.03884660474716 +- 1.357870090244534
MSE no validation: 20.70579271604938 +- 5.026246163328767
r2 no training: 0.8351980911330458 +- 0.012630917380617364
r2 no validation: 0.7565437120824658 +- 0.03503397711452812


In [12]:
print("R2 no Teste: {}".format(r2_score(y_test, knn.predict(X_test))))
print("MSE no Teste: {}".format(mean_squared_error(y_test, knn.predict(X_test))))

R2 no Teste: 0.4950376724477026
MSE no Teste: 41.11829411764707


O modelo de KNN está mais overfitado que o modelo linear, pois o delta entre métricas de treino e validação ficou maior.

# 2.4 - Modelo de árvore

In [13]:
dt = DecisionTreeRegressor(max_depth=3, min_samples_leaf=5, random_state=0)

cross_validation(dt, kf, X_train, y_train)

MSE no training: 16.386198585799406 +- 0.6778800457501502
MSE no validation: 26.6914033845927 +- 6.090645773615034
r2 no training: 0.807246350717681 +- 0.008701552820878922
r2 no validation: 0.6749472274089795 +- 0.09997891320406922


In [14]:
print("R2 no Teste: {}".format(r2_score(y_test, dt.predict(X_test))))
print("MSE no Teste: {}".format(mean_squared_error(y_test, dt.predict(X_test))))

R2 no Teste: 0.5176066160280801
MSE no Teste: 39.28054027062085


# 2.5 - Modelo de ensemble

In [15]:
rf = RandomForestRegressor(n_estimators=500, max_depth=3, min_samples_leaf=5, random_state=0)

cross_validation(rf, kf, X_train, y_train)

MSE no training: 11.373107019491664 +- 0.5697216919889382
MSE no validation: 16.94212619999738 +- 3.613170463514426
r2 no training: 0.8661262127760668 +- 0.008489118103769578
r2 no validation: 0.7966737518859056 +- 0.043336716909563956


In [16]:
print("R2 no Teste: {}".format(r2_score(y_test, rf.predict(X_test))))
print("MSE no Teste: {}".format(mean_squared_error(y_test, rf.predict(X_test))))

R2 no Teste: 0.6079529967193826
MSE no Teste: 31.923775516035974


O Random Forest parece ser o mais promissor. Suas métrica são superiores aos outros modelos, porém ainda overfita bastante. Vamos tentar parametrizar melhor.

### Feature Importance

In [17]:
pd.DataFrame(
    data = rf.feature_importances_, 
    index = X_train.columns,
    columns=['Importância']
).reset_index().sort_values(by='Importância', ascending=False)

,index,Importância
12,LSTAT,0.640601
5,RM,0.258053
0,CRIM,0.048877
7,DIS,0.020170
10,PTRATIO,0.012864
4,NOX,0.008175
11,B,0.003072
2,INDUS,0.002772
9,TAX,0.002328
6,AGE,0.002321


# 3 - Hiperparametrização

## 3.1- GridSearchCV

In [18]:
# Definindo grid para busca
params_grid = {
    'n_estimators': [100, 300, 600, 1000],
    'max_features': ['auto'],
    'max_depth': [2,3],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True]
}

# criando nosso modelo de grid search
gridsearch_model = GridSearchCV(
    estimator = rf, 
    param_grid = params_grid,
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5,
    verbose = 2
)

In [19]:
%time gridsearch_model.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
CPU times: user 3.15 s, sys: 170 ms, total: 3.32 s
Wall time: 2min 23s


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=3, min_samples_leaf=5,
                                             n_estimators=500, random_state=0),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [2, 3],
                         'max_features': ['auto'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 300, 600, 1000]},
             scoring='neg_mean_absolute_error', verbose=2)

In [20]:
print('Melhores parâmetros: ')
print(gridsearch_model.best_params_)

Melhores parâmetros: 
{'bootstrap': True, 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 600}


In [21]:
best_grid_rf = gridsearch_model.best_estimator_

## 3.2 - RandomSearchCV

In [22]:
# Definindo grid para busca
params_grid = {
    'n_estimators': [100, 300, 600, 1000],
    'max_features': ['auto'],
    'max_depth': [2,3],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True]
}

# criando nosso modelo de grid search
randomsearch_model = RandomizedSearchCV(
    estimator = rf, 
    param_distributions = params_grid,
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5,
    verbose = 2
)

In [23]:
%time randomsearch_model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: user 1.27 s, sys: 23.4 ms, total: 1.29 s
Wall time: 25.2 s


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(max_depth=3,
                                                   min_samples_leaf=5,
                                                   n_estimators=500,
                                                   random_state=0),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [2, 3],
                                        'max_features': ['auto'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 300, 600, 1000]},
                   scoring='neg_mean_absolute_error', verbose=2)

In [24]:
print('Melhores parâmetros: ')
print(randomsearch_model.best_params_)

Melhores parâmetros: 
{'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 3, 'bootstrap': True}


In [25]:
best_random_rf = randomsearch_model.best_estimator_

## 3.3 - Resultados

### GridSearchCV

In [26]:
y_pred_test = best_grid_rf.predict(X_test)

print("R2 no conjunto de testes: {}".format(r2_score(y_test, y_pred_test)))
print("Mean squared error no conjunto de testes: {}".format(mean_squared_error(y_test, y_pred_test)))

R2 no conjunto de testes: 0.6192167737243803
Mean squared error no conjunto de testes: 31.006583736577696


In [27]:
best_grid_rf.score(X_train, y_train)

0.8644858137977192

In [28]:
cross_validation(best_grid_rf, kf, X_train, y_train)

MSE no training: 11.113872059373865 +- 0.5600235815238439
MSE no validation: 16.88456016260671 +- 3.5066504235171
r2 no training: 0.8691717412083634 +- 0.008393589005525585
r2 no validation: 0.7971288043354241 +- 0.043633880561391204


### RandomSearchCV

In [29]:
y_pred_test = best_random_rf.predict(X_test)

print("R2 no conjunto de testes: {}".format(r2_score(y_test, y_pred_test)))
print("Mean squared error no conjunto de testes: {}".format(mean_squared_error(y_test, y_pred_test)))

R2 no conjunto de testes: 0.6192167737243803
Mean squared error no conjunto de testes: 31.006583736577696


In [30]:
best_random_rf.score(X_train, y_train)

0.8644858137977192

In [31]:
cross_validation(best_random_rf, kf, X_train, y_train)

MSE no training: 11.113872059373865 +- 0.5600235815238439
MSE no validation: 16.88456016260671 +- 3.5066504235171
r2 no training: 0.8691717412083634 +- 0.008393589005525585
r2 no validation: 0.7971288043354241 +- 0.043633880561391204


# 4 - Seleção de variáveis

### Recursive Feature Elimination

In [32]:
selector = RFE(rf, n_features_to_select=6, step=1)
selector = selector.fit(X_train, y_train)

In [33]:
mascara_best_features = selector.support_

X_train_rfe = X_train.loc[:, mascara_best_features].copy()
X_test_rfe = X_test.loc[:, mascara_best_features].copy()

In [34]:
cross_validation(rf, kf, X_train_rfe, y_train)

MSE no training: 11.665915780101601 +- 0.6454513239937704
MSE no validation: 16.799226123776492 +- 3.846162236338197
r2 no training: 0.8626996112657814 +- 0.009026419940651304
r2 no validation: 0.7983273998273799 +- 0.0461260867376871


In [35]:
print("R2 no conjunto de testes: {}".format(r2_score(y_test, rf.predict(X_test_rfe))))
print("Mean squared error no conjunto de testes: {}".format(mean_squared_error(y_test, rf.predict(X_test_rfe))))

R2 no conjunto de testes: 0.610362894321099
Mean squared error no conjunto de testes: 31.727541316029193


### Boruta

[link](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a) com a técnica

In [36]:
from boruta import BorutaPy

# inicializando boruta
boruta = BorutaPy(
    estimator = rf, 
    n_estimators = 'auto',
    max_iter = 100,
    random_state = 0
)

# Selecionando as variáveis
boruta.fit(np.array(X_train), np.array(y_train))

### print results
green_area = X_train.columns[boruta.support_].to_list()
blue_area = X_train.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

features in the green area: ['CRIM', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'PTRATIO', 'LSTAT']
features in the blue area: ['TAX']


In [37]:
X_train_boruta = X_train[green_area].copy()
X_test_boruta = X_test[green_area].copy()

In [38]:
cross_validation(rf, kf, X_train_boruta, y_train)

MSE no training: 11.444855815803582 +- 0.5708118091560461
MSE no validation: 16.835443526170693 +- 3.654608159594576
r2 no training: 0.8652371712019168 +- 0.009164146573972225
r2 no validation: 0.797828369113866 +- 0.044445910304314946


In [39]:
print("R2 no conjunto de testes: {}".format(r2_score(y_test, rf.predict(X_test_boruta))))
print("Mean squared error no conjunto de testes: {}".format(mean_squared_error(y_test, rf.predict(X_test_boruta))))

R2 no conjunto de testes: 0.6053992657945666
Mean squared error no conjunto de testes: 32.13172183902785


# 5 - Conclusões

In [40]:
pd.DataFrame(
    {
        'MSE_training':[20.17, 19.96, 14.03, 16.38, 11.37],
        'MSE_validation':['-', 21.99, 20.70, 26.69, 16.94],
        'MSE_test':[38.53, 39.96, 41.11, 39.28, 31.92],
        'R2_training':[0.76, 0.76, 0.83, 0.80, 0.86],
        'R2_validation':['-',0.73, 0.75, 0.67, 0.79],
        'R2_test':[0.52, 0.50, 0.49, 0.51, 0.60]
    },
    index=['OLS','Modelo_linear', 'Modelo_distancia', 'Modelo_arvore', 'Modelo_ensemble']
)

,MSE_training,MSE_validation,MSE_test,R2_training,R2_validation,R2_test
OLS,20.17,-,38.53,0.76,-,0.52
Modelo_linear,19.96,21.99,39.96,0.76,0.73,0.50
Modelo_distancia,14.03,20.7,41.11,0.83,0.75,0.49
Modelo_arvore,16.38,26.69,39.28,0.80,0.67,0.51
Modelo_ensemble,11.37,16.94,31.92,0.86,0.79,0.60




- testamos alguns modelos:
    - linear regression
    - knn
    - decision tree
    - random forest
- avaliamos em validação cruzada
- testamos otimização de hiperparâmetros
    - grid search
    - random search
- testamos seleção de variáveis
    - RFE
    - Boruta
    
- Concluimos que o melhor modelo encontrado ainda assim overfita
- Dentro os modelos testados, o que menos overfitou foi o RF
- Portanto, na escolha do melhor modelo, escolheria o RF